# Group Project - KSI data - Classification problem

***Target Column***
ACCLASS<br>
Required to transform into binary (0, 1):<br>
'Fatal' --> 1, <br>
'Non-Fatal Injury' --> 0, <br>
'Property Damage Only' --> 0, <br>
***5 nan value from this column, we can consider to drop them***



below columns need to fill values:
'PEDESTRIAN', 'CYCLIST', 'AUTOMOBILE', 'MOTORCYCLE', 'TRUCK',
'TRSN_CITY_VEH', 'EMERG_VEH', 'PASSENGER', 'SPEEDING', 'AG_DRIV',
'REDLIGHT', 'ALCOHOL', 'DISABILITY'
fill Nan as No, and transform to 0, 1
(Default they are Yes, Nan values)
ROAD_CLASS fill most freq value
DISTRICT fill most freq value

Questionable column:
CYCCOND: multi categories, fill Nan as most freq value??


From the dataset, below columns are unnecessary:
ObjectId, HEIGHBOURHOOD_158, HEIGHBOURHOOD_140, CYCLISTYPE(too much categories and too much Nan value),<br>
PEDCOND(too much categories and too much Nan value), PEDACT(too much categories and too much Nan value),<br>
PEDTYPE (too much categories and too much Nan value), DRICOND ('other' included, means it is not a accuracy value), DRIVACT ('other' included, means it is not a accuracy value), MANOEUVER('other' included, means it is not a accuracy value)<br>
FATAL_NO, INVTYPE, DATE, YEAR, ACCNUM, INDEX_, STREET1, STREET2, OFFSET, X, Y,INJURY



In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
dataset_path = r'dataset\KSI.csv'

df = pd.read_csv(dataset_path)

In [3]:
df.head(5)

X             Y   INDEX_    ACCNUM  YEAR  \
0 -8.844611e+06  5.412414e+06  3387730  892658.0  2006   
1 -8.844611e+06  5.412414e+06  3387731  892658.0  2006   
2 -8.816480e+06  5.434843e+06  3388101  892810.0  2006   
3 -8.829728e+06  5.419071e+06  3389067  893184.0  2006   
4 -8.816480e+06  5.434843e+06  3388102  892810.0  2006   

                     DATE  TIME          STREET1         STREET2 OFFSET  ...  \
0  2006/03/11 05:00:00+00   852       BLOOR ST W     DUNDAS ST W    NaN  ...   
1  2006/03/11 05:00:00+00   852       BLOOR ST W     DUNDAS ST W    NaN  ...   
2  2006/03/11 05:00:00+00   915  MORNINGSIDE AVE  SHEPPARD AVE E    NaN  ...   
3  2006/01/01 05:00:00+00   236     WOODBINE AVE     O CONNOR DR    NaN  ...   
4  2006/03/11 05:00:00+00   915  MORNINGSIDE AVE  SHEPPARD AVE E    NaN  ...   

  AG_DRIV REDLIGHT  ALCOHOL  DISABILITY  HOOD_158 NEIGHBOURHOOD_158 HOOD_140  \
0     Yes      NaN      NaN         NaN        88   High Park North       88   
1     Yes      NaN      NaN         NaN        88   High Park North       88   
2     Yes      Yes      NaN         NaN       146      Malvern East      132   
3     Yes      NaN      Yes         NaN        60  Woodbine-Lumsden       60   
4     Yes      Yes      NaN         NaN       146      Malvern East      132   

       NEIGHBOURHOOD_140 DIVISION ObjectId  
0   High Park North (88)      D11        1  
1   High Park North (88)      D11        2  
2          Malvern (132)      D42        3  
3  Woodbine-Lumsden (60)      D55        4  
4          Malvern (132)      D42        5  

[5 rows x 57 columns]

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18194 entries, 0 to 18193
Data columns (total 57 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   X                  18194 non-null  float64
 1   Y                  18194 non-null  float64
 2   INDEX_             18194 non-null  int64  
 3   ACCNUM             13264 non-null  float64
 4   YEAR               18194 non-null  int64  
 5   DATE               18194 non-null  object 
 6   TIME               18194 non-null  int64  
 7   STREET1            18194 non-null  object 
 8   STREET2            16510 non-null  object 
 9   OFFSET             3402 non-null   object 
 10  ROAD_CLASS         17818 non-null  object 
 11  DISTRICT           18089 non-null  object 
 12  WARDNUM            17332 non-null  float64
 13  LATITUDE           18194 non-null  float64
 14  LONGITUDE          18194 non-null  float64
 15  LOCCOORD           18099 non-null  object 
 16  ACCLOC             127

In [5]:
df.columns.values

array(['X', 'Y', 'INDEX_', 'ACCNUM', 'YEAR', 'DATE', 'TIME', 'STREET1',
       'STREET2', 'OFFSET', 'ROAD_CLASS', 'DISTRICT', 'WARDNUM',
       'LATITUDE', 'LONGITUDE', 'LOCCOORD', 'ACCLOC', 'TRAFFCTL',
       'VISIBILITY', 'LIGHT', 'RDSFCOND', 'ACCLASS', 'IMPACTYPE',
       'INVTYPE', 'INVAGE', 'INJURY', 'FATAL_NO', 'INITDIR', 'VEHTYPE',
       'MANOEUVER', 'DRIVACT', 'DRIVCOND', 'PEDTYPE', 'PEDACT', 'PEDCOND',
       'CYCLISTYPE', 'CYCACT', 'CYCCOND', 'PEDESTRIAN', 'CYCLIST',
       'AUTOMOBILE', 'MOTORCYCLE', 'TRUCK', 'TRSN_CITY_VEH', 'EMERG_VEH',
       'PASSENGER', 'SPEEDING', 'AG_DRIV', 'REDLIGHT', 'ALCOHOL',
       'DISABILITY', 'HOOD_158', 'NEIGHBOURHOOD_158', 'HOOD_140',
       'NEIGHBOURHOOD_140', 'DIVISION', 'ObjectId'], dtype=object)

# Exploration <br>
Use below code to display categrial data and null counts

In [13]:
print(df['DRIVCOND'].value_counts())
print(df['DRIVCOND'].isnull().sum())

DRIVCOND
Normal                                5847
Inattentive                           1581
Unknown                               1100
Medical or Physical Disability         177
Had Been Drinking                      163
Ability Impaired, Alcohol Over .08     126
Ability Impaired, Alcohol              121
Other                                   52
Fatigue                                 51
Ability Impaired, Drugs                 20
Name: count, dtype: int64
8951


# Determine necessary columns

In [14]:
#Since 5 rows are missing target values (ACCLASS), we will remove them
df = df.dropna(subset=['ACCLASS'])

#We will remove the columns that are not useful for our model
meaningless_columns = ['INDEX_', 'ACCNUM', 'YEAR', 'DATE', 'TIME', 'STREET1',
                       'STREET2', 'OFFSET', 'FATAL_NO', 'NEIGHBOURHOOD_158', 'NEIGHBOURHOOD_140',
                       'ObjectId']

too_much_missing = ['PEDTYPE','CYCACT', 'CYCLISTYPE', 'PEDACT', 'CYCCOND']

#We will remove the columns with duplicate information
# X and Y are the same as LONGITUDE and LATITUDE
# VEHTYPE, PEDCOND, DRIVCOND duplicated because there are categorical columns for the same information
duplicate_columns = ['X', 'Y', 'VEHTYPE', 'PEDCOND', 'DRIVCOND']

#Columns need to fill missing values as 0
fill_zero_columns = ['WARDNUM']

#columns need to fill Nan values
binary_map = {np.nan: 'No'}
fill_nan_columns = ['PEDESTRIAN', 'CYCLIST', 'AUTOMOBILE', 'MOTORCYCLE', 'TRUCK',
                    'TRSN_CITY_VEH', 'EMERG_VEH', 'PASSENGER', 'SPEEDING', 'AG_DRIV',
                    'REDLIGHT', 'ALCOHOL', 'DISABILITY']

#columns which contain categorical data
categorical_columns = ['LIGHT', 'INVAGE', 'IMPACTYPE', 'RDSFCOND', 
                       'DISTRICT', 'DRIVACT', 'INITDIR', 'ROAD_CLASS', 'TRAFFCTL', 
                       'ACCLOC', 'VISIBILITY','INVTYPE', 'MANOEUVER']



In [15]:
df[categorical_columns].isnull().sum()

LIGHT            0
INVAGE           0
IMPACTYPE        4
RDSFCOND        25
DISTRICT       105
DRIVACT       8948
INITDIR       5051
ROAD_CLASS     376
TRAFFCTL        34
ACCLOC        5450
VISIBILITY      20
INVTYPE         16
MANOEUVER     7659
dtype: int64

In [16]:
#Columns can be try to exclude or include
try_columns = ['DIVISION', 'LOCCOORD', 'INJURY']

In [17]:
#Find remaining columns
remaining_columns = list(set(df.columns.values) - set(meaningless_columns) - set(too_much_missing) - set(duplicate_columns) - set(fill_zero_columns) - set(fill_nan_columns) - set(categorical_columns) - set(try_columns))
print(remaining_columns)

['HOOD_140', 'ACCLASS', 'LATITUDE', 'LONGITUDE', 'HOOD_158']


In [18]:
#Check remaining columns for missing values
df[remaining_columns].isnull().sum()

HOOD_140     0
ACCLASS      0
LATITUDE     0
LONGITUDE    0
HOOD_158     0
dtype: int64

In [29]:
#Make a copy of the dataframe
df_copy = df.copy()

#Drop meaningless columns
df_copy = df_copy.drop(columns=meaningless_columns)
#Drop too much missing columns
df_copy = df_copy.drop(columns=too_much_missing)
#Drop duplicate columns
df_copy = df_copy.drop(columns=duplicate_columns)
#Fill missing values with 'No'
df_copy[fill_nan_columns] = df_copy[fill_nan_columns].fillna(value='No')
#Fill missing values with 0
df_copy[fill_zero_columns] = df_copy[fill_zero_columns].fillna(value=0)

#drop y column
df_copy = df_copy.drop(columns=['ACCLASS'])

#Set y as target variable
y = df['ACCLASS']

In [33]:
#train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_copy, y, test_size=0.8, random_state=58)

In [34]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(3637, 34) (14552, 34) (3637,) (14552,)


In [31]:
y.replace({'Property Damage Only': 'Non-Fatal Injury'}, inplace=True)

In [32]:
y.value_counts()

ACCLASS
Non-Fatal Injury    15616
Fatal                2573
Name: count, dtype: int64

In [36]:
#tartget values are imbalanced, we will use SMOTE to balance the target values
from imblearn import over_sampling
smote = over_sampling.SMOTE(random_state=58)
X_train, y_train = smote.fit_resample(X_train, y_train)


ImportError: cannot import name '_MissingValues' from 'sklearn.utils._param_validation' (c:\ProgramData\anaconda3\Lib\site-packages\sklearn\utils\_param_validation.py)

In [ ]:
#Create a pipeline to transform the data

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

#Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='most_frequent')

#Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

#Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, remaining_columns),
        ('cat', categorical_transformer, categorical_columns)
    ])

#Define the model
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, random_state=0)

#Bundle preprocessing and modeling code in a pipeline
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)
                     ])

#Preprocessing of training data, fit model
clf.fit(df_copy, y)

In [19]:
#Mapping for target column
print(df['ACCLASS'].value_counts())

# Only Fatal and Non-Fatal from our prediction
# 1 for Fatal, 0 for Non-Fatal
data_map = {
    'Fatal': 1, 
    'Non-Fatal Injury': 0, 
    'Property Damage Only': 0
    }

ACCLASS
Non-Fatal Injury        15599
Fatal                    2573
Property Damage Only       17
Name: count, dtype: int64


In [24]:
df.describe()

X             Y        INDEX_        ACCNUM          YEAR  \
count  1.819400e+04  1.819400e+04  1.819400e+04  1.326400e+04  18194.000000   
mean  -8.838345e+06  5.420748e+06  3.818870e+07  4.248444e+08   2012.934869   
std    1.162533e+04  8.682160e+03  3.726463e+07  1.065503e+09      4.754258   
min   -8.865305e+06  5.402162e+06  3.363207e+06  2.530100e+04   2006.000000   
25%   -8.846591e+06  5.413242e+06  5.391370e+06  1.021229e+06   2009.000000   
50%   -8.838448e+06  5.419556e+06  7.644612e+06  1.197308e+06   2012.000000   
75%   -8.829671e+06  5.427813e+06  8.078261e+07  1.365020e+06   2017.000000   
max   -8.807929e+06  5.443099e+06  8.170606e+07  4.008024e+09   2022.000000   

               TIME       WARDNUM      LATITUDE     LONGITUDE    FATAL_NO  \
count  18194.000000  1.733200e+04  18194.000000  18194.000000  827.000000   
mean    1362.615917  2.521028e+03     43.710459    -79.396201   29.073761   
std      630.816048  1.844803e+05      0.056369      0.104432   17.803627   
min        0.000000  1.000000e+00     43.589678    -79.638390    1.000000   
25%      920.000000  7.000000e+00     43.661727    -79.470280   14.000000   
50%     1450.000000  1.300000e+01     43.702745    -79.397132   28.000000   
75%     1850.000000  2.200000e+01     43.756345    -79.318286   42.000000   
max     2359.000000  1.716222e+07     43.855445    -79.122974   78.000000   

           ObjectId  
count  18194.000000  
mean    9097.500000  
std     5252.299734  
min        1.000000  
25%     4549.250000  
50%     9097.500000  
75%    13645.750000  
max    18194.000000